In [10]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import openpyxl
import dbfread  


### test cargar db

In [11]:
prueba = funciones.subir_dbf()
print(prueba['IT_ITOT'])


C:/Users/zickd/Downloads/INDICADORV54.DBF
       ORD_ID ORD_NUMERO  EXP_ID   OS_NOMBRE                      NOMBRE SEXO  \
0       25549  Amb-33444    8032  OSOETSYLRA        INEXISTENTE,AFILIADO    M   
1       25550  Amb-33445    8033  OSOETSYLRA        INEXISTENTE,AFILIADO    M   
2       25639  Amb-33536    8037     VISITAR        GONZALEZ,MARíA BELEN    F   
3       25641  Amb-33562    8052  OSOETSYLRA        PEON,RENZO VALENTINO    M   
4       25642  Amb-33569    8053  OSOETSYLRA            SOSA,AXEL HERNAN    M   
...       ...        ...     ...         ...                         ...  ...   
38726   49537      54380   12598  OSOETSYLRA   BURGUEÑO,MARIA DEL CARMEN    F   
38727   49538  Amb-59196   13132  OSOETSYLRA    IBARRA GARAY,URIEL ELIAS    M   
38728   49539  Amb-59197   13133  OSOETSYLRA         OJEDA,LAUTARO URIEL    M   
38729   49540  Amb-59198   13134  OSOETSYLRA    IBARRA GARAY,URIEL ELIAS    M   
38730   49541  Amb-59199   13135  OSOETSYLRA  PACHECO ALIAGA,HOMERO

#### leer DBF simpe

In [3]:
class MyFieldParser(dbfread.FieldParser):

    # Sobrescribir el método parseN
    def parseN(self, field, data):
        # Si el campo es "importe", intentar convertir el valor a cadena
        if field.name == "importe":
            try:
                return str(data)
            # Si hay un error, devolver el valor original
            except ValueError:
                return data
        # Si el campo no es "importe", devolver el valor original
        else:
            return data
        
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            dbf =  dbfread.DBF( ruta_archivo, load=True, parserclass=MyFieldParser)
            indicadores = pd.DataFrame(dbf)
            #indicadores = pd.read_excel(ruta_archivo, engine='openpyxl')
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/zickd/Downloads/INDICADORESV53.DBF' mode='r' encoding='cp1252'>


In [12]:
prueba['IT_ITOT'].tail

<bound method NDFrame.tail of 0          58000.00
1         148264.34
2          74232.00
3         142093.00
4           7339.00
            ...    
38726       7441.47
38727    1192000.00
38728     606720.00
38729    1404000.00
38730     159174.00
Name: IT_ITOT, Length: 38731, dtype: float64>

In [13]:
print(prueba.columns)

Index(['ORD_ID', 'ORD_NUMERO', 'EXP_ID', 'OS_NOMBRE', 'NOMBRE', 'SEXO', 'EDAD',
       'LOCAAFI', 'ORD_FSOL', 'ORD_FEC', 'NOMSOL', 'ORD_ORI', 'ORI_NOM',
       'LOCAORI', 'ORD_AREA', 'NOMENCLA', 'IT_COD', 'NOM_NOM', 'CANTIDAD',
       'IT_ITOT', 'MOT_NOM', 'NOMGTS', 'TEXP_NOMBR', 'PLAN_NOMBR', 'MONODROGA',
       'ACCION', 'PROV_AFI', 'IT_OBS', 'TIPOORIGEN', 'AGENTE_GTS', 'COB_ESP',
       'DOC_ID', 'FECHAALTA', 'ORD_PERI'],
      dtype='object')


### modelado

In [14]:
diccionario = {
'CONSULTA':[420301,
420132,
420125,
420134,
420126,
420127,
420130,
420121,
420117,
420129,
420110,
420104,
420145,
420105,
420102,
420109,
420128,
420108,
420101,
420103,
1,
420120,
3,
420133,
420107,
420112,
420143,
420135,
420106,
101,
420303,
420144,
420141]}

# creo la funcion para iterar el diciccionario

def aplicar_categoria(codigo):
    for categoria, codigos in diccionario.items():
        if codigo in codigos:
            return categoria
    return "practica"


# Crear campo pais
#indicadores['PAIS'] = 'ARGENTINA' + ',' + indicadores['PROV_AFI'] + ',' + indicadores['LOCAAFI']
prueba['PAIS'] = 'ARGENTINA' + ',' + prueba['PROV_AFI'] + ',' + prueba['LOCAAFI']
# separar el campo patologias y el campo coberturas especiales
prueba["COB_ESP"] = prueba["COB_ESP"].str.replace("\\(\\d+\\/\\d+\\/\\d+-\\d+\\/\\d+\\/\\d+\\)|\\/\\d+", "")
            

C:\Users\zickd\AppData\Local\Temp\ipykernel_14748\788434183.py:49: FutureWarning: The default value of regex will change from True to False in a future version.
  prueba["COB_ESP"] = prueba["COB_ESP"].str.replace("\\(\\d+\\/\\d+\\/\\d+-\\d+\\/\\d+\\/\\d+\\)|\\/\\d+", "")


### aplicar diccionario y crear nuevo campo practicas_consultas

In [15]:
#indicadores['CONSULTAS_PRACTICAS']=indicadores['IT_COD'].apply(aplicar_categoria)
prueba['CONSULTAS_PRACTICAS']=prueba['IT_COD'].apply(aplicar_categoria)

#print(indicadores[['IT_COD','CONSULTAS_PRACTICAS']])
#save_path = filedialog.askdirectory()
#test
#indicadores2 = indicadores[[
indicadores2 = prueba[[
"ORD_ID",
"ORD_NUMERO",
"ORD_PERI",
"EXP_ID",
"OS_NOMBRE",
"NOMBRE",
"SEXO",
"EDAD",
"LOCAAFI",
"ORD_FSOL",
"ORD_FEC",
"NOMSOL",
"ORD_ORI",
"ORI_NOM",
"LOCAORI",
"ORD_AREA",
"NOMENCLA",
"IT_COD",
"NOM_NOM",
"CONSULTAS_PRACTICAS",
"CANTIDAD",
"IT_ITOT",
"MOT_NOM",
"NOMGTS",
"TEXP_NOMBR",
"PLAN_NOMBR",
"MONODROGA",
"ACCION",
"PAIS",
"PROV_AFI",
"IT_OBS",
"TIPOORIGEN",
"AGENTE_GTS",
"COB_ESP",
"DOC_ID",
"FECHAALTA"
]]
indicadores2.columns= indicadores2.columns.str.lower()
#indicadores2['it_itot'] = indicadores2['it_itot'].replace('b':'',   regex=True)
#indicadores2.to_excel(save_path + '\indicadoresv5.xlsx', engine='openpyxl' ,index = False )
#indicadores2.to_csv(save_path + '\indicadoresv5.csv' ,index = False )
print(indicadores2['it_itot'])

0          58000.00
1         148264.34
2          74232.00
3         142093.00
4           7339.00
            ...    
38726       7441.47
38727    1192000.00
38728     606720.00
38729    1404000.00
38730     159174.00
Name: it_itot, Length: 38731, dtype: float64


### SUBIR EL EXCEL CORREGIDO

In [5]:
#cargar primero el excel o csv
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            indicadores3 = pd.read_excel(ruta_archivo)
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(indicadores)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/zickd/Downloads/indicadoresv5.xlsx' mode='r' encoding='cp1252'>
       ORD_ID ORD_NUMERO  EXP_ID   OS_NOMBRE                  NOMBRE SEXO  \
0       25549  Amb-33444    8032  OSOETSYLRA    INEXISTENTE,AFILIADO    M   
1       25550  Amb-33445    8033  OSOETSYLRA    INEXISTENTE,AFILIADO    M   
2       25639  Amb-33536    8037     VISITAR    GONZALEZ,MARíA BELEN    F   
3       25641  Amb-33562    8052  OSOETSYLRA    PEON,RENZO VALENTINO    M   
4       25642  Amb-33569    8053  OSOETSYLRA        SOSA,AXEL HERNAN    M   
...       ...        ...     ...         ...                     ...  ...   
27148   42333      43593   11950     VISITAR  GIMENEZ,LEANDRO JAVIER    M   
27149   42333      43593   11950     VISITAR  GIMENEZ,LEANDRO JAVIER    M   
27150   42333      43593   11950     VISITAR  GIMENEZ,LEANDRO JAVIER    M   
27151   42333      43593   11950     VISITAR  GIMENEZ,LEANDRO JAVIER    M   
27152   42333      43593   11950     VISITAR  GIMENEZ,L

In [16]:
indicadores2['it_itot'] = indicadores2['it_itot'].astype(str).str.replace('.',',')

C:\Users\zickd\AppData\Local\Temp\ipykernel_14748\114153593.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  indicadores2['it_itot'] = indicadores2['it_itot'].astype(str).str.replace('.',',')


### GUARDAR EL DATAFRAME EN UNA HOJA DE GSHEET

In [6]:
import pygsheets
import pandas as pd

path='C:\\Users\zickd\\OneDrive\Documentos\\repos_git\\signomedico-398902-d596060414cb.json'
gc=pygsheets.authorize(service_account_file=path)
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/10IKRk1MISpvgmRxH1pj_63Kt8PMEEJGvJp8I6unsa3c/edit?usp=drive_link')
wk1=sh[0] # first worksheet 
#wk1.clear() # to remove all data from sheet
wk1.set_dataframe(indicadores2,(1,1), extend=True, copy_head=True)

In [8]:
print(dbfread.__version__)
indicadores2['cob_esp']



2.0.7


0         
1         
2         
3         
4         
        ..
38726     
38727     
38728     
38729     
38730     
Name: cob_esp, Length: 38731, dtype: object